In [9]:
# NOTE: Some modifications to multimodal-transformer source code have been performed but are not tracked by version control.

In [5]:
! pip install multimodal-transformers

  Using cached transformers-3.1.0-py3-none-any.whl (884 kB)
  Using cached tokenizers-0.8.1rc2-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.2
    Uninstalling tokenizers-0.10.2:
      Successfully uninstalled tokenizers-0.10.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.6.0.dev0
    Uninstalling transformers-4.6.0.dev0:
      Successfully uninstalled transformers-4.6.0.dev0


In [15]:
import json
import re
import sys
import pandas as pd
import numpy as np

## Preprocessing

In [1]:
import dataloader

In [2]:
train_df = dataloader.load_data(data_file = '../data/train.data.jsonl', label_file = '../data/train.label.json', perform_stemming = False)
dev_df = dataloader.load_data(data_file = '../data/dev.data.jsonl', label_file = '../data/dev.label.json', perform_stemming = False)
test_df = dataloader.load_data(data_file = '../data/test.data.jsonl', label_file = None, perform_stemming = False)

In [7]:
test_df['text'][0]

'5 people have been able to get out of sydney cafe during hostage situation: fucking terrorists ...   ,      ,     ,    .  5 people have been able to get out of sydney cafe during hostage situation: her fingers look broken. 5 otages libres 5 people have been able to get out of sydney cafe : definitely adding that tweet to my 5 people have been able to get out of sydney cafe during hostage situation:   cuba usa stop showing their faces, they are going to have enough trouble without forever being linked with this event i sure hope the hostage taker gets out. feet first and room temp. 5 people have been able to get out of sydney cafe during hostage situation: dear mr. swat team guy, you may want to choose a long rifle for a more precise shot. this is not csi or miami vice. radical islam again spreading terror.  muslims are fanatical zealots p2 isis syndeysiege anyone see the connection yet? how long r free and democratic nations going to allow terrorists and state sponsors of terror dicta

## Multimodal BERT

In [8]:
from dataclasses import dataclass, field
import json
import logging
import os
from typing import Optional

from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    EvalPrediction,
    set_seed
)
from transformers.training_args import TrainingArguments

from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular

logging.basicConfig(level=logging.INFO)
os.environ['COMET_MODE'] = 'DISABLED'

In [9]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """
    model_name_or_path: str = field(
      metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
      default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )


@dataclass
class MultimodalDataTrainingArguments:
    """
    Arguments pertaining to how we combine tabular features
    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    data_path: str = field(metadata={
                            'help': 'the path to the csv file containing the dataset'
                        })
    column_info_path: str = field(
      default=None,
      metadata={
          'help': 'the path to the json file detailing which columns are text, categorical, numerical, and the label'
    })

    column_info: dict = field(
      default=None,
      metadata={
          'help': 'a dict referencing the text, categorical, numerical, and label columns'
                  'its keys are text_cols, num_cols, cat_cols, and label_col'
    })

    categorical_encode_type: str = field(default='ohe',
                                        metadata={
                                            'help': 'sklearn encoder to use for categorical data',
                                            'choices': ['ohe', 'binary', 'label', 'none']
                                        })
    numerical_transformer_method: str = field(default='yeo_johnson',
                                            metadata={
                                                'help': 'sklearn numerical transformer to preprocess numerical data',
                                                'choices': ['yeo_johnson', 'box_cox', 'quantile_normal', 'none']
                                            })
    task: str = field(default="classification",
                    metadata={
                        "help": "The downstream training task",
                        "choices": ["classification", "regression"]
                    })

    mlp_division: int = field(default=4,
                            metadata={
                                'help': 'the ratio of the number of '
                                        'hidden dims in a current layer to the next MLP layer'
                            })
    combine_feat_method: str = field(default='individual_mlps_on_cat_and_numerical_feats_then_concat',
                                    metadata={
                                        'help': 'method to combine categorical and numerical features, '
                                                'see README for all the method'
                                    })
    mlp_dropout: float = field(default=0.1,
                              metadata={
                                'help': 'dropout ratio used for MLP layers'
                              })
    numerical_bn: bool = field(default=True,
                              metadata={
                                  'help': 'whether to use batchnorm on numerical features'
                              })
    use_simple_classifier: str = field(default=True,
                                      metadata={
                                          'help': 'whether to use single layer or MLP as final classifier'
                                      })
    mlp_act: str = field(default='relu',
                        metadata={
                            'help': 'the activation function to use for finetuning layers',
                            'choices': ['relu', 'prelu', 'sigmoid', 'tanh', 'linear']
                        })
    gating_beta: float = field(default=0.2,
                              metadata={
                                  'help': "the beta hyperparameters used for gating tabular data "
                                          "see https://www.aclweb.org/anthology/2020.acl-main.214.pdf"
                              })

    def __post_init__(self):
        assert self.column_info != self.column_info_path
        if self.column_info is None and self.column_info_path:
            with open(self.column_info_path, 'r') as f:
                self.column_info = json.load(f)

In [26]:
text_cols = ['text']
cat_cols = ['question_mark', 'contains_url', 'contains_media', 'contains_profile_background_image', 'verified', 'geo_enabled', 'has_description']
numerical_cols = ['retweet_count', 'favorite_count', 'number_urls','statuses_count', 'listed_count', 'reputation_score_1', 'reputation_score_2', 'favourites_count','length_description','follow_tweets']

column_info_dict = {
    'text_cols': text_cols,
    'num_cols': numerical_cols,
    'cat_cols': cat_cols,
    'label_col': 'label',
    'label_list': [0, 1]
}


model_args = ModelArguments(
    model_name_or_path='bert-base-uncased'
)

data_args = MultimodalDataTrainingArguments(
    data_path='.',
    combine_feat_method='gating_on_cat_and_num_feats_then_sum',
    column_info=column_info_dict,
    task='classification'
)

training_args = TrainingArguments(
    output_dir="./logs/model_name",
    logging_dir="./logs/runs",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    evaluate_during_training=True,
    logging_steps=25,
    eval_steps=250,
    dataloader_drop_last=True
)

set_seed(training_args.seed)

In [27]:
tokenizer_path_or_name = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
print('Specified tokenizer: ', tokenizer_path_or_name)

# Tokens automatically converted to lower_case
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path_or_name,
    cache_dir=model_args.cache_dir
)

Specified tokenizer:  bert-base-uncased


In [28]:
# Get Datasets
train_dataset, dev_dataset, test_dataset = load_data_from_folder(train_df, dev_df, test_df,
    data_args.column_info['text_cols'],
    tokenizer,
    label_col=data_args.column_info['label_col'],
    label_list=data_args.column_info['label_list'],
    categorical_cols=data_args.column_info['cat_cols'],
    numerical_cols=data_args.column_info['num_cols'],
    sep_text_token_str=tokenizer.sep_token
)

INFO:multimodal_transformers.data.data_utils:9 numerical columns
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
INFO:multimodal_transformers.data.data_utils:20 categorical columns
INFO:multimodal_transformers.data.data_utils:9 numerical columns
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
INFO:multimodal_transformers.data.load_data:Text

In [29]:
num_labels = len(np.unique(train_dataset.labels))
num_labels

2

In [30]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
tabular_config = TabularConfig(num_labels=num_labels,
                               cat_feat_dim=train_dataset.cat_feats.shape[1],
                               numerical_feat_dim=train_dataset.numerical_feats.shape[1],
                               **vars(data_args))
config.tabular_config = tabular_config

In [31]:
model = AutoModelWithTabular.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        config=config,
        cache_dir=model_args.cache_dir
    )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertWithTabular: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertWithTabular from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertWithTabular from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertWithTabular were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifi

In [32]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import (
    auc,
    precision_recall_curve,
    roc_auc_score,
    f1_score,
    confusion_matrix,
    matthews_corrcoef,
)

def calc_classification_metrics(p: EvalPrediction):
    pred_labels = np.argmax(p.predictions, axis=1)
    pred_scores = softmax(p.predictions, axis=1)[:, 1]
    labels = p.label_ids
    if len(np.unique(labels)) == 2:  # binary classification
        roc_auc_pred_score = roc_auc_score(labels, pred_scores)
        precisions, recalls, thresholds = precision_recall_curve(labels,
                                                                pred_scores)
        fscore = (2 * precisions * recalls) / (precisions + recalls)
        fscore[np.isnan(fscore)] = 0
        ix = np.argmax(fscore)
        threshold = thresholds[ix].item()
        pr_auc = auc(recalls, precisions)
        tn, fp, fn, tp = confusion_matrix(labels, pred_labels, labels=[0, 1]).ravel()
        result = {'roc_auc': roc_auc_pred_score,
                'threshold': threshold,
                'pr_auc': pr_auc,
                'recall': recalls[ix].item(),
                'precision': precisions[ix].item(), 'f1': fscore[ix].item(),
                'tn': tn.item(), 'fp': fp.item(), 'fn': fn.item(), 'tp': tp.item()
                }
    else:
        acc = (pred_labels == labels).mean()
        f1 = f1_score(y_true=labels, y_pred=pred_labels)
        result = {
          "acc": acc,
          "f1": f1,
          "acc_and_f1": (acc + f1) / 2,
          "mcc": matthews_corrcoef(labels, pred_labels)
        }

    return result

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=calc_classification_metrics
)

In [33]:
%%time
trainer.train()

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1160 [00:00<?, ?it/s]

{'loss': 0.4349016571044922, 'learning_rate': 3.472413793103448e-05, 'epoch': 0.021551724137931036, 'step': 25}
{'loss': 0.3521604156494141, 'learning_rate': 3.450862068965517e-05, 'epoch': 0.04310344827586207, 'step': 50}
{'loss': 0.49650787353515624, 'learning_rate': 3.4293103448275864e-05, 'epoch': 0.06465517241379311, 'step': 75}
{'loss': 0.4685893249511719, 'learning_rate': 3.4077586206896555e-05, 'epoch': 0.08620689655172414, 'step': 100}
{'loss': 0.26140182495117187, 'learning_rate': 3.386206896551724e-05, 'epoch': 0.10775862068965517, 'step': 125}
{'loss': 0.6234982299804688, 'learning_rate': 3.364655172413793e-05, 'epoch': 0.12931034482758622, 'step': 150}
{'loss': 0.41747283935546875, 'learning_rate': 3.343103448275862e-05, 'epoch': 0.15086206896551724, 'step': 175}
{'loss': 0.50378662109375, 'learning_rate': 3.321551724137931e-05, 'epoch': 0.1724137931034483, 'step': 200}
{'loss': 0.710081787109375, 'learning_rate': 3.3e-05, 'epoch': 0.1939655172413793, 'step': 225}
{'loss':

Evaluation:   0%|          | 0/72 [00:00<?, ?it/s]

{'eval_loss': 0.5579873754953345, 'eval_roc_auc': 0.9078163771712159, 'eval_threshold': 0.013931883499026299, 'eval_pr_auc': 0.7808453934714581, 'eval_recall': 0.8548387096774194, 'eval_precision': 0.7464788732394366, 'eval_f1': 0.7969924812030075, 'eval_tn': 355, 'eval_fp': 35, 'eval_fn': 48, 'eval_tp': 138, 'epoch': 0.21551724137931033, 'step': 250}
{'loss': 0.5334515380859375, 'learning_rate': 3.256896551724138e-05, 'epoch': 0.23706896551724138, 'step': 275}
{'loss': 0.5242626953125, 'learning_rate': 3.235344827586207e-05, 'epoch': 0.25862068965517243, 'step': 300}
{'loss': 0.562830810546875, 'learning_rate': 3.213793103448276e-05, 'epoch': 0.2801724137931034, 'step': 325}
{'loss': 0.423704833984375, 'learning_rate': 3.192241379310345e-05, 'epoch': 0.3017241379310345, 'step': 350}
{'loss': 0.4186285400390625, 'learning_rate': 3.170689655172414e-05, 'epoch': 0.3232758620689655, 'step': 375}
{'loss': 0.477371826171875, 'learning_rate': 3.149137931034483e-05, 'epoch': 0.344827586206896

...

{'eval_loss': 0.4838169237870413, 'eval_roc_auc': 0.9511579818031431, 'eval_threshold': 0.9618346095085144, 'eval_pr_auc': 0.8592406871803954, 'eval_recall': 0.8709677419354839, 'eval_precision': 0.8393782383419689, 'eval_f1': 0.8548812664907651, 'eval_tn': 355, 'eval_fp': 35, 'eval_fn': 23, 'eval_tp': 163, 'epoch': 4.741379310344827, 'step': 5500}
{'loss': 0.0588916015625, 'learning_rate': 0.0, 'epoch': 4.762931034482759, 'step': 5525}
{'loss': 0.328466796875, 'learning_rate': 0.0, 'epoch': 4.7844827586206895, 'step': 5550}
{'loss': 0.2914208984375, 'learning_rate': 0.0, 'epoch': 4.806034482758621, 'step': 5575}
{'loss': 0.2051708984375, 'learning_rate': 0.0, 'epoch': 4.827586206896552, 'step': 5600}
{'loss': 0.116826171875, 'learning_rate': 0.0, 'epoch': 4.849137931034483, 'step': 5625}
{'loss': 0.14458984375, 'learning_rate': 0.0, 'epoch': 4.870689655172414, 'step': 5650}
{'loss': 0.2078515625, 'learning_rate': 0.0, 'epoch': 4.892241379310345, 'step': 5675}
{'loss': 0.1364453125, 'l

Evaluation:   0%|          | 0/72 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


{'eval_loss': 0.48394241781271474, 'eval_roc_auc': 0.9511993382961125, 'eval_threshold': 0.9596925377845764, 'eval_pr_auc': 0.8552557144186422, 'eval_recall': 0.8709677419354839, 'eval_precision': 0.8393782383419689, 'eval_f1': 0.8548812664907651, 'eval_tn': 355, 'eval_fp': 35, 'eval_fn': 23, 'eval_tp': 163, 'epoch': 4.956896551724138, 'step': 5750}
{'loss': 0.12205078125, 'learning_rate': 0.0, 'epoch': 4.978448275862069, 'step': 5775}
{'loss': 0.297763671875, 'learning_rate': 0.0, 'epoch': 5.0, 'step': 5800}
CPU times: user 1h 7min 22s, sys: 21min 55s, total: 1h 29min 17s
Wall time: 1h 29min 44s


TrainOutput(global_step=5800, training_loss=0.29739531418372844)

In [21]:
# Load TensorBoard notebook extension
%load_ext tensorboard

In [22]:
%tensorboard --logdir ./logs/runs

In [34]:
trainer.save_model("./multimodal_bert/multimodal_bert_v34")

### Inference

In [35]:
# Need to reload trainer because drop_last = True was set during training.
pred_trained = Trainer(
    model=model
)

In [36]:
result = pred_trained.predict(test_dataset=test_dataset).predictions

Prediction:   0%|          | 0/73 [00:00<?, ?it/s]

In [37]:
from scipy.special import softmax

pred_labels = np.argmax(result, axis=1)
pred_scores = softmax(result, axis=1)[:, 1]

In [38]:
predicted_labels = [dataloader.convert_prediction(pred) for pred in pred_labels]

In [39]:
output = pd.DataFrame({'id':test_df.id,'target':predicted_labels})
output

,id,target
0,544382249178001408,rumour
1,525027317551079424,rumour
2,544273220128739329,rumour
3,499571799764770816,rumour
4,552844104418091008,rumour
...,...,...
576,553581227165642752,rumour
577,552816302780579840,rumour
578,580350000074457088,rumour
579,498584409055174656,rumour


In [40]:
submission = pd.Series(output.target.values,index=output.id).to_dict()
with open('test-output.json', 'w') as f:
    json.dump(submission, f)